In [4]:
import numpy as np
import pandas as pd
import re
from pprint import pprint
clinical_df = pd.read_excel("20200414.xlsx", sheet_name=0)
library_df = pd.read_excel("20200414.xlsx", sheet_name=1)

In [5]:
sampleId_count_clinical = clinical_df["样本编号"].value_counts()
sampleId_unique_clinical = sampleId_count_clinical.index[sampleId_count_clinical==1]
print("total sampleId in clinical_df: {}; unique sampleId(count=1) in clinical_df: {}".format(clinical_df.shape[0], sampleId_unique_clinical.shape[0]))

total sampleId in clinical_df: 2357; unique sampleId(count=1) in clinical_df: 2065


In [6]:
library_df.loc[:, "虚拟入库样本名"] = library_df.loc[:, "虚拟入库样本名"].fillna(method='ffill')
library_df.loc[:, "pooling比例"] = library_df.loc[:, "pooling比例"].fillna(method='ffill')	
library_df.loc[:, "取样ng"] = library_df.loc[:, "取样ng"].fillna(method='ffill')	
library_df.loc[:, "体积ul"] = library_df.loc[:, "体积ul"].fillna(method='ffill')	
library_df.loc[:, "杂交探针"] = library_df.loc[:, "杂交探针"].fillna(method='ffill')	
library_df.loc[:, "杂交时间h"] = library_df.loc[:, "杂交时间h"].fillna(method='ffill')	
library_df.loc[:, "PostPCR循环数"] = library_df.loc[:, "PostPCR循环数"].fillna(method='ffill')	
library_df.loc[:, "PostPCR浓度ng/ul"] = library_df.loc[:, "PostPCR浓度ng/ul"].fillna(method='ffill')	
library_df.loc[:, "洗脱体积"] = library_df.loc[:, "洗脱体积"].fillna(method='ffill')	
library_df.loc[:, "文库质控"] = library_df.loc[:, "文库质控"].fillna(method='ffill')	
library_df.loc[:, "文库名称"] = library_df.loc[:, "文库名称"].fillna(method='ffill')
library_df = library_df.fillna('无')
library_df["华大样本编号P"] = library_df["华大样本编号"].str.slice_replace(2, 3, repl='P')
sampleId_count_libaray = library_df["华大样本编号P"].value_counts()
sampleId_unique_libaray = sampleId_count_libaray.index[sampleId_count_libaray==1]
print("total sampleId in library_df: {}; unique sampleId(count=1) in library_df: {}".format(library_df.shape[0], sampleId_unique_libaray.shape[0]))

total sampleId in library_df: 2197; unique sampleId(count=1) in clinical_df: 1408


In [7]:
sampleId_unique_overlap = sampleId_unique_libaray[sampleId_unique_libaray.isin(sampleId_unique_clinical)]
print("overlap unique sampleId(count=1) in clinical_df and library_df: {}".format(sampleId_unique_overlap.shape[0]))

overlap unique sampleId(count=1) in clinical_df and library_df: 1105


In [8]:
library_df_filt = library_df.loc[library_df["华大样本编号P"].isin(sampleId_unique_overlap)]
library_df_filt["建库编号"] = library_df["华大样本编号"].str.replace(re.compile(r'$'), '_1_m1')
library_df_filt["DNA提取编号"] = library_df["华大样本编号"].str.replace(re.compile(r'$'), '_1')
count = {}
libraryId_list = []
for dnaId in library_df_filt.loc[:, "DNA提取编号"]:
    dnaId = str(dnaId)
    count[dnaId] = count[dnaId]+1 if dnaId in count else 1
    libraryId_list.append("{}_m{}".format(dnaId, count[dnaId]))

clinical_df_filt = clinical_df.loc[clinical_df["样本编号"].isin(sampleId_unique_overlap)]
with pd.ExcelWriter("library.20200424.xlsx") as writer:
    library_df_filt.to_excel(writer, sheet_name="建库表(clinicalDf_overlap_libraryDf)", index=False)
    clinical_df_filt.to_excel(writer, sheet_name="临床信息表(clinicalDf_overlap_libraryDf)", index=False)


In [7]:
for i in count:
    if count[i]>1:
        print("k: {}; v: {}".format(i, count[i]))

In [6]:
filenames = []
df_list = []

In [7]:
# 处理"BGI健康人 20190905.xlsx"
test_df = pd.read_excel("BGI健康人 20190905.xlsx", sheet_name="1 临床信息表")
test_df_filt = test_df.loc[test_df["样本编号"].isin(sampleId_unique_overlap)]
filenames.append("BGI健康人 20190905.xlsx")
df_list.append(test_df_filt)

In [8]:
print("shape of test_df: {}; shape of test_df_filt: {}; ".format(test_df.shape, test_df_filt.shape))

shape of test_df: (96, 19); shape of test_df_filt: (96, 19); 


In [9]:
# 处理"肺癌.xlsx"
test_df = pd.read_excel("肺癌.xlsx", sheet_name=0)
test_df_filt = test_df.loc[test_df["华大编号"].isin(sampleId_unique_overlap)]
print("shape of test_df: {}; shape of test_df_filt: {}; ".format(test_df.shape, test_df_filt.shape))

shape of test_df: (129, 71); shape of test_df_filt: (43, 71); 


In [10]:
def joinField(series_):
    s=""
    series_new = series_.copy(deep=True)
    series_new.fillna("无", inplace=True)
    for idx in range(series_new.shape[0]):
        v=series_new[idx] if series_new[idx]!="" else "无"
        s = s+"; {}: {}".format(series_new.index[idx], v)
    return s[2:]

In [11]:
# test_df_filt.ix[test_df_filt.index[0]].fillna("无")

In [12]:
test_df_filt.loc[:,"Gender"] = test_df_filt.loc[:,"Gender"].str.replace("F", "女")
test_df_filt.loc[:,"Gender"] = test_df_filt.loc[:,"Gender"].str.replace("M", "男")
data = pd.DataFrame({
    "样本编号": test_df_filt.loc[:,"华大编号"],
    "姓名": test_df_filt.loc[:,"姓名"],
    "性别": test_df_filt.loc[:,"Gender"],
    "年龄": test_df_filt.loc[:,"Age"],
    "住院号": test_df_filt.loc[:,"patient_hospital_ID"],
    "癌种": test_df_filt.loc[:,"病理"],
    "分期": test_df_filt.loc[:,"Stage"],
    "诊断": test_df_filt.loc[:,"病理诊断"],
    "诊断备注": [ joinField(test_df_filt.ix[idx, ["TNM", "病理诊断", "位置", "随访记录", "FU_last", "Status", "Recurrence", "DFS", "FU_DFS", "OS", "FU_OS", "Smoking", "是否复发（0否，1是）", "局部复发/远处转移", "肿瘤复发时间部位", "无瘤生存时间（月）", "复发后治疗", "是否死亡（0存活，1死亡）", "死亡时间（月）", "总存活时间", "死亡是否与肿瘤相关（0无关，1相关）", "慢性病史", "既往肿瘤病史", "吸烟史（无吸烟史=0，有吸烟史记录包年数）", "是否戒烟（无吸烟史=0；未戒烟=0；戒烟=1）", "戒烟时长（x月，或x年）小于1月不记录", "肺癌家族史（1有，写出具体关系； 0无）"]]) for idx in test_df_filt.index ],
    "采样日期": test_df_filt.loc[:,"手术日期"],
    "医院编号": ["无" for x in test_df_filt.index],
    "科室": ["无" for x in test_df_filt.index],
    "血浆管数": test_df_filt.loc[:, "血标本"].str.extract(r'p\*(\d+)', flags=re.IGNORECASE, expand=True).fillna(0).iloc[:,0],
    "癌旁组织": test_df_filt.loc[:,"癌旁组织标本(管数)N"],
    "癌组织": test_df_filt.loc[:,"肿瘤标本(管数)T"],
    "白细胞": test_df_filt.loc[:, "血标本"].str.extract(r'wbc\*(\d+)', flags=re.IGNORECASE, expand=True).fillna(0).iloc[:,0],
    "粪便": [0 for x in test_df_filt.index],
    "寄送日期": [np.nan for x in test_df_filt.index],
    "备注": test_df_filt.loc[:,"备注"]
    })

filenames.append("肺癌.xlsx")
df_list.append(data)

In [13]:
# 处理"福建肠癌.xlsx"
test_df = pd.read_excel("福建肠癌.xlsx", sheet_name=0)
test_df_filt = test_df.loc[test_df["华大编号"].isin(sampleId_unique_overlap)]
print("shape of test_df: {}; shape of test_df_filt: {}; ".format(test_df.shape, test_df_filt.shape))

shape of test_df: (24, 28); shape of test_df_filt: (0, 28); 


In [14]:
# 处理"广州南医五院第二批样本 20191119.xlsx"
test_df = pd.read_excel("广州南医五院第二批样本 20191119.xlsx", sheet_name=0)
test_df_filt = test_df.loc[test_df["华大编号"].isin(sampleId_unique_overlap)]
print("shape of test_df: {}; shape of test_df_filt: {}; ".format(test_df.shape, test_df_filt.shape))

shape of test_df: (182, 27); shape of test_df_filt: (66, 27); 


In [15]:
data = pd.DataFrame({
    "样本编号": test_df_filt.loc[:,"华大编号"],
    "姓名": test_df_filt.loc[:,"姓名"],
    "性别": test_df_filt.loc[:,"性别"],
    "年龄": test_df_filt.loc[:,"年龄"],
    "住院号": test_df_filt.loc[:,"原始样本编号"],
    "癌种": ["无" for x in test_df_filt.index],
    "分期": ["无" for x in test_df_filt.index],
    "诊断": ["无" for x in test_df_filt.index],
    "诊断备注": ["无" for x in test_df_filt.index],
    "采样日期": [np.nan for x in test_df_filt.index],
    "医院编号": ["广州南医五院" for x in test_df_filt.index],
    "科室": ["无" for x in test_df_filt.index],
    "血浆管数": [1 if pd.isna(test_df_filt.loc[x,"另一管血的编号"]) else 2 for x in test_df_filt.index],
    "癌旁组织": [0 for x in test_df_filt.index],
    "癌组织": [0 for x in test_df_filt.index],
    "白细胞": [0 if pd.isna(test_df_filt.loc[x,"另一管血的编号"]) else 1 for x in test_df_filt.index],
    "粪便": [0 for x in test_df_filt.index],
    "寄送日期": test_df_filt.loc[:,"送检日期"],
    "备注": [ joinField(test_df_filt.ix[idx, ["采血管编号", "另一管血的编号", "送样批次", "备注"]]) for idx in test_df_filt.index ]
    })

filenames.append("广州南医五院第二批样本 20191119.xlsx")
df_list.append(data)

In [16]:
# 处理"广州南医五院第一批样本 20190905.xlsx"
test_df = pd.read_excel("广州南医五院第一批样本 20190905.xlsx", sheet_name="1 临床信息表")
test_df_filt = test_df.loc[test_df["样本编号"].isin(sampleId_unique_overlap)]
print("shape of test_df: {}; shape of test_df_filt: {}; ".format(test_df.shape, test_df_filt.shape))

shape of test_df: (102, 19); shape of test_df_filt: (49, 19); 


In [17]:
filenames.append("广州南医五院第一批样本 20190905.xlsx")
df_list.append(test_df_filt)

In [18]:
# 处理"湖南省肿瘤医院结直肠癌 20191107.xlsx"
test_df = pd.read_excel("湖南省肿瘤医院结直肠癌 20191107.xlsx", sheet_name="1 临床信息表")
test_df_filt = test_df.loc[test_df["样本编号"].isin(sampleId_unique_overlap)]
print("shape of test_df: {}; shape of test_df_filt: {}; ".format(test_df.shape, test_df_filt.shape))

shape of test_df: (100, 19); shape of test_df_filt: (53, 19); 


In [19]:
filenames.append("湖南省肿瘤医院结直肠癌 20191107.xlsx")
df_list.append(test_df_filt)

In [20]:
# 处理"华大肠癌项目-样本信息登记表.xlsx"
test_df1 = pd.read_excel("华大肠癌项目-样本信息登记表.xlsx", sheet_name="肠癌")
test_df1_filt = test_df1.loc[test_df1["华大编号"].isin(sampleId_unique_overlap)]
print("shape of test_df1: {}; shape of test_df1_filt: {}; ".format(test_df1.shape, test_df1_filt.shape))
test_df2 = pd.read_excel("华大肠癌项目-样本信息登记表.xlsx", sheet_name="健康人")
test_df2_filt = test_df2.loc[test_df2["全血编号"].isin(sampleId_unique_overlap)]
print("shape of test_df2: {}; shape of test_df2_filt: {}; ".format(test_df2.shape, test_df2_filt.shape))


shape of test_df1: (128, 35); shape of test_df1_filt: (0, 35); 
shape of test_df2: (15, 18); shape of test_df2_filt: (0, 18); 


In [21]:
# 处理"南通肝癌.xlsx"
test_df = pd.read_excel("南通肝癌.xlsx", sheet_name=0)
test_df_filt = test_df.loc[test_df["华大编号"].isin(sampleId_unique_overlap)]
print("shape of test_df: {}; shape of test_df_filt: {}; ".format(test_df.shape, test_df_filt.shape))

shape of test_df: (9, 28); shape of test_df_filt: (0, 28); 


In [22]:
# 处理"上海肺癌.xlsx"
test_df = pd.read_excel("上海肺癌.xlsx", sheet_name=0)
test_df_filt = test_df.loc[test_df["华大编号"].isin(sampleId_unique_overlap)]
print("shape of test_df: {}; shape of test_df_filt: {}; ".format(test_df.shape, test_df_filt.shape))

shape of test_df: (235, 26); shape of test_df_filt: (62, 26); 


In [23]:
data = pd.DataFrame({
    "样本编号": test_df_filt.loc[:,"华大编号"],
    "姓名": test_df_filt.loc[:,"姓名"],
    "性别": test_df_filt.loc[:,"性别"],
    "年龄": test_df_filt.loc[:,"年龄"],
    "住院号": test_df_filt.loc[:,"原始样本编号"],
    "癌种": test_df_filt.loc[:,"癌种"],
    "分期": ["无" for x in test_df_filt.index],
    "诊断": test_df_filt.loc[:,"病理"],
    "诊断备注": ["无" for x in test_df_filt.index],
    "采样日期": test_df_filt.loc[:,"采样日期"],
    "医院编号": ["无" for x in test_df_filt.index],
    "科室": ["无" for x in test_df_filt.index],
    "血浆管数": [1 for x in test_df_filt.index],
    "癌旁组织": [0 for x in test_df_filt.index],
    "癌组织": [0 for x in test_df_filt.index],
    "白细胞": [0 for x in test_df_filt.index],
    "粪便": [0 for x in test_df_filt.index],
    "寄送日期": [np.nan for x in test_df_filt.index],
    "备注": test_df_filt.loc[:,"送样批次"]
    })

filenames.append("上海肺癌.xlsx")
df_list.append(data)

In [24]:
# 处理"上海复旦中山胰腺癌 20191107.xlsx"
test_df = pd.read_excel("上海复旦中山胰腺癌 20191107.xlsx", sheet_name="1 临床信息表")
test_df_filt = test_df.loc[test_df["样本编号"].isin(sampleId_unique_overlap)]
print("shape of test_df: {}; shape of test_df_filt: {}; ".format(test_df.shape, test_df_filt.shape))

shape of test_df: (121, 19); shape of test_df_filt: (118, 19); 


In [25]:
filenames.append("上海复旦中山胰腺癌 20191107.xlsx")
df_list.append(test_df_filt)

In [26]:
# 处理"上海肝癌.xlsx"
test_df = pd.read_excel("上海肝癌.xlsx", sheet_name=0)
test_df_filt = test_df.loc[test_df["华大编号"].isin(sampleId_unique_overlap)]
print("shape of test_df: {}; shape of test_df_filt: {}; ".format(test_df.shape, test_df_filt.shape))

shape of test_df: (426, 28); shape of test_df_filt: (212, 28); 


In [27]:
data = pd.DataFrame({
    "样本编号": test_df_filt.loc[:,"华大编号"],
    "姓名": test_df_filt.loc[:,"姓名"],
    "性别": test_df_filt.loc[:,"性别"],
    "年龄": test_df_filt.loc[:,"年龄"],
    "住院号": test_df_filt.loc[:,"原始样本编号"],
    "癌种": test_df_filt.loc[:,"癌种"],
    "分期": ["无" for x in test_df_filt.index],
    "诊断": ["无" for x in test_df_filt.index],
    "诊断备注": ["无" for x in test_df_filt.index],
    "采样日期": test_df_filt.loc[:,"采样日期"],
    "医院编号": ["无" for x in test_df_filt.index],
    "科室": ["无" for x in test_df_filt.index],
    "血浆管数": [1 for x in test_df_filt.index],
    "癌旁组织": [0 for x in test_df_filt.index],
    "癌组织": [0 for x in test_df_filt.index],
    "白细胞": [0 for x in test_df_filt.index],
    "粪便": [0 for x in test_df_filt.index],
    "寄送日期": [np.nan for x in test_df_filt.index],
    "备注": test_df_filt.loc[:,"送样批次"]
    })

filenames.append("上海肝癌.xlsx")
df_list.append(data)

In [28]:
# 处理"上海肝硬化.xlsx"
test_df = pd.read_excel("上海肝硬化.xlsx", sheet_name=0)
test_df_filt = test_df.loc[test_df["华大编号"].isin(sampleId_unique_overlap)]
print("shape of test_df: {}; shape of test_df_filt: {}; ".format(test_df.shape, test_df_filt.shape))

shape of test_df: (41, 22); shape of test_df_filt: (20, 22); 


In [29]:
data = pd.DataFrame({
    "样本编号": test_df_filt.loc[:,"华大编号"],
    "姓名": test_df_filt.loc[:,"姓名"],
    "性别": test_df_filt.loc[:,"性别"],
    "年龄": test_df_filt.loc[:,"年龄"],
    "住院号": test_df_filt.loc[:,"原始样本编号"],
    "癌种": test_df_filt.loc[:,"癌种"],
    "分期": ["无" for x in test_df_filt.index],
    "诊断": ["无" for x in test_df_filt.index],
    "诊断备注": ["无" for x in test_df_filt.index],
    "采样日期": test_df_filt.loc[:,"采样日期"],
    "医院编号": ["无" for x in test_df_filt.index],
    "科室": ["无" for x in test_df_filt.index],
    "血浆管数": [1 for x in test_df_filt.index],
    "癌旁组织": [0 for x in test_df_filt.index],
    "癌组织": [0 for x in test_df_filt.index],
    "白细胞": [0 for x in test_df_filt.index],
    "粪便": [0 for x in test_df_filt.index],
    "寄送日期": [np.nan for x in test_df_filt.index],
    "备注": test_df_filt.loc[:,"送样批次"]
    })

filenames.append("上海肝硬化.xlsx")
df_list.append(data)

In [30]:
# 处理"上海组织.xlsx"
test_df = pd.read_excel("上海组织.xlsx", sheet_name=0)
test_df_filt = test_df.loc[test_df["华大编号"].isin(sampleId_unique_overlap)]
print("shape of test_df: {}; shape of test_df_filt: {}; ".format(test_df.shape, test_df_filt.shape))

shape of test_df: (60, 16); shape of test_df_filt: (0, 16); 


In [31]:
# 处理"胃癌组织.xlsx"
test_df = pd.read_excel("胃癌组织.xlsx", sheet_name=0)
test_df_filt = test_df.loc[test_df["华大虚拟编号"].isin(sampleId_unique_overlap)]
print("shape of test_df: {}; shape of test_df_filt: {}; ".format(test_df.shape, test_df_filt.shape))

shape of test_df: (171, 14); shape of test_df_filt: (0, 14); 


In [32]:
# 处理"中大六院肠癌早筛-样本信息登记表(回顾性两批)L.xlsx"
test_df = pd.read_excel("中大六院肠癌早筛-样本信息登记表(回顾性两批)L.xlsx", sheet_name=0)
test_df_filt = test_df.loc[test_df["华大编号"].isin(sampleId_unique_overlap)]
print("shape of test_df: {}; shape of test_df_filt: {}; ".format(test_df.shape, test_df_filt.shape))

shape of test_df: (41, 30); shape of test_df_filt: (17, 30); 


In [33]:
data = pd.DataFrame({
    "样本编号": test_df_filt.loc[:,"华大编号"],
    "姓名": test_df_filt.loc[:,"姓名"],
    "性别": test_df_filt.loc[:,"性别"],
    "年龄": test_df_filt.loc[:,"年龄"],
    "住院号": ["无" for x in test_df_filt.index],
    "癌种": test_df_filt.loc[:,"诊断"],
    "分期": ["无" for x in test_df_filt.index],
    "诊断": test_df_filt.loc[:,"诊断"],
    "诊断备注": [ joinField(test_df_filt.ix[idx, ["TNM", "TTNNMM"]]) for idx in test_df_filt.index ],
    "采样日期": test_df_filt.loc[:,"采样日期"],
    "医院编号": test_df_filt.loc[:,"医院编号"],
    "科室": ["无" for x in test_df_filt.index],
    "血浆管数": [1 for x in test_df_filt.index],
    "癌旁组织": [0 for x in test_df_filt.index],
    "癌组织": [0 for x in test_df_filt.index],
    "白细胞": [0 for x in test_df_filt.index],
    "粪便": [0 for x in test_df_filt.index],
    "寄送日期": test_df_filt.loc[:,"寄送日期"],
    "备注": ["无" for x in test_df_filt.index]
    })

filenames.append("中大六院肠癌早筛-样本信息登记表(回顾性两批)L.xlsx")
df_list.append(data)

In [34]:
# 处理"中大六院肠癌早筛-样本信息登记表2020-3-31.xlsx"
test_df = pd.read_excel("中大六院肠癌早筛-样本信息登记表2020-3-31.xlsx", sheet_name=0)
test_df_filt = test_df.loc[test_df["华大编号"].isin(sampleId_unique_overlap)]
print("shape of test_df: {}; shape of test_df_filt: {}; ".format(test_df.shape, test_df_filt.shape))

shape of test_df: (254, 28); shape of test_df_filt: (190, 28); 


In [35]:
data = pd.DataFrame({
    "样本编号": test_df_filt.loc[:,"华大编号"],
    "姓名": test_df_filt.loc[:,"姓名"],
    "性别": test_df_filt.loc[:,"性别"],
    "年龄": test_df_filt.loc[:,"年龄"],
    "住院号": test_df_filt.loc[:,"全血编号"],
    "癌种": test_df_filt.loc[:,"诊断"],
    "分期": ["无" for x in test_df_filt.index],
    "诊断": test_df_filt.loc[:,"诊断"],
    "诊断备注": [ joinField(test_df_filt.ix[idx, ["T", "N", "M", "TNM"]]) for idx in test_df_filt.index ],
    "采样日期": test_df_filt.loc[:,"采样日期"],
    "医院编号": test_df_filt.loc[:,"医院编号"],
    "科室": ["无" for x in test_df_filt.index],
    "血浆管数": [1 for x in test_df_filt.index],
    "癌旁组织": [0 for x in test_df_filt.index],
    "癌组织": [0 for x in test_df_filt.index],
    "白细胞": [0 for x in test_df_filt.index],
    "粪便": [0 for x in test_df_filt.index],
    "寄送日期": test_df_filt.loc[:,"寄送日期"],
    "备注": ["无" for x in test_df_filt.index]
    })

filenames.append("中大六院肠癌早筛-样本信息登记表2020-3-31.xlsx")
df_list.append(data)

In [36]:
# 处理"中山大学肿瘤医院-肠癌.xlsx"
test_df1 = pd.read_excel("中山大学肿瘤医院-肠癌.xlsx", sheet_name="肠癌")
test_df1_filt = test_df1.loc[test_df1["华大编号"].isin(sampleId_unique_overlap)]
print("shape of test_df1: {}; shape of test_df1_filt: {}; ".format(test_df1.shape, test_df1_filt.shape))
test_df2 = pd.read_excel("中山大学肿瘤医院-肠癌.xlsx", sheet_name="健康人")
test_df2_filt = test_df2.loc[test_df2["全血编号"].isin(sampleId_unique_overlap)]
print("shape of test_df2: {}; shape of test_df2_filt: {}; ".format(test_df2.shape, test_df2_filt.shape))

shape of test_df1: (128, 34); shape of test_df1_filt: (0, 34); 
shape of test_df2: (15, 6); shape of test_df2_filt: (0, 6); 


In [37]:
# 处理"中山肝癌MRD-20191113.xlsx"
test_df = pd.read_excel("中山肝癌MRD-20191113.xlsx", sheet_name="1 临床信息表")
test_df_filt = test_df.loc[test_df["样本编号"].isin(sampleId_unique_overlap)]
print("shape of test_df: {}; shape of test_df_filt: {}; ".format(test_df.shape, test_df_filt.shape))

shape of test_df: (112, 19); shape of test_df_filt: (112, 19); 


In [38]:
test_df_filt.loc[:,"备注"] = ["无" for x in test_df_filt.index]
filenames.append("中山肝癌MRD-20191113.xlsx")
df_list.append(test_df_filt)


In [39]:
print("len of filenames: {}; len of df_list: {}".format(len(filenames), len(df_list)))

len of filenames: 12; len of df_list: 12


In [41]:
with pd.ExcelWriter("clinical.20200426.xlsx") as writer:
    for idx in range(len(filenames)):
        df_list[idx].to_excel(writer, sheet_name=filenames[idx], index=False)


In [42]:
df_list[0].head

<bound method NDFrame.head of           样本编号   姓名 性别  年龄      住院号  癌种  分期      诊断  \
0   19P3758198  李春萱  女  57  19P0501 NaN NaN  BGI健康人   
1   19P3758199   蔡流  男  60  19P0502 NaN NaN  BGI健康人   
2   19P3758200   钟燕  女  57  19P0503 NaN NaN  BGI健康人   
3   19P3758201  彭小中  女  51  19P0504 NaN NaN  BGI健康人   
4   19P3758202  韩桂芬  女  60  19P0505 NaN NaN  BGI健康人   
..         ...  ... ..  ..      ...  ..  ..     ...   
91  19P3758293   漆红  女  62  19P0596 NaN NaN  BGI健康人   
92  19P3758294  蔡立凤  女  51  19P0597 NaN NaN  BGI健康人   
93  19P3758295  陈从献  男  55  19P0598 NaN NaN  BGI健康人   
94  19P3758296   刘琳  女  56  19P0599 NaN NaN  BGI健康人   
95  19P3758297   吴洪  男  59  19P0600 NaN NaN  BGI健康人   

                            诊断备注       采样日期  医院编号  科室  血浆管数  癌旁组织  癌组织  白细胞  \
0             肿瘤患病史: 无, 家族肿瘤史: 无 2019-04-24   NaN NaN     1     0    0    0   
1             肿瘤患病史: 无, 家族肿瘤史: 无 2019-04-24   NaN NaN     1     0    0    1   
2             肿瘤患病史: 无, 家族肿瘤史: 无 2019-04-24   NaN NaN     1     0    0  

In [45]:
sample_id_df_list = {}
for idx in range(len(filenames)):
    df = df_list[idx]
    for idx2 in range(df.shape[0]):
        row = df.index[idx2]
        if df.loc[row, "样本编号"] in sample_id_df_list:
            sample_id_df_list[df.loc[row, "样本编号"]].append(filenames[idx])
        else:
            sample_id_df_list[df.loc[row, "样本编号"]] = [filenames[idx]]

for k in sample_id_df_list:
    if len(sample_id_df_list[k])>1:
        print("key: {}; value: {}".format(k, sample_id_df_list[k]))

In [46]:
sample_id_df_list

9P5298106': ['上海肝癌.xlsx'],
 '19P5127839': ['上海肝癌.xlsx'],
 '19P4178176': ['上海肝癌.xlsx'],
 '19P0138950': ['上海肝癌.xlsx'],
 '19P0138943': ['上海肝癌.xlsx'],
 '19P5790156': ['上海肝癌.xlsx'],
 '19P0138946': ['上海肝癌.xlsx'],
 '19P0165020': ['上海肝癌.xlsx'],
 '19P0156423': ['上海肝癌.xlsx'],
 '19P7247305': ['上海肝癌.xlsx'],
 '19P0138941': ['上海肝癌.xlsx'],
 '19P5790161': ['上海肝癌.xlsx'],
 '19P7247313': ['上海肝癌.xlsx'],
 '19P5790196': ['上海肝癌.xlsx'],
 '19P5127863': ['上海肝癌.xlsx'],
 '19P5298095': ['上海肝癌.xlsx'],
 '19P0138945': ['上海肝癌.xlsx'],
 '19P0138942': ['上海肝癌.xlsx'],
 '19P5127820': ['上海肝癌.xlsx'],
 '19P0165021': ['上海肝癌.xlsx'],
 '19P0138944': ['上海肝癌.xlsx'],
 '19P7247280': ['上海肝癌.xlsx'],
 '19P6714090': ['上海肝癌.xlsx'],
 '19P5790167': ['上海肝癌.xlsx'],
 '19P5790193': ['上海肝癌.xlsx'],
 '19P0156426': ['上海肝癌.xlsx'],
 '19P0156427': ['上海肝癌.xlsx'],
 '19P5127818': ['上海肝癌.xlsx'],
 '19P5790184': ['上海肝癌.xlsx'],
 '19P7247335': ['上海肝癌.xlsx'],
 '19P5790195': ['上海肝癌.xlsx'],
 '19P7247344': ['上海肝癌.xlsx'],
 '19P5790155': ['上海肝癌.xlsx'],
 '19P5298099'